In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
from pykrx import stock
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind
from sklearn.linear_model import LassoCV, RidgeCV
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from tqdm import tqdm
ticker_list = stock.get_market_ticker_list(market="KOSDAQ")

#ticker_list=[]
# precision_svm 값과 종목코드를 저장할 빈 리스트
results_list = []

# 각 종목에 대해 반복
for ticker in tqdm(ticker_list, desc="Processing"):
    try:
        ticker_name = stock.get_market_ticker_name(ticker)
        start_date = "2020-03-20"
        end_date = "2024-03-20"
        # 주어진 기간 동안의 일별 거래량 정보 가져오기
        df = stock.get_market_ohlcv_by_date(fromdate=start_date, todate=end_date, ticker=ticker)


        def calculate_macd(df, short_window=12, long_window=26, signal_window=9):
            """MACD 및 MACD 신호 계산"""
            df['EMA_short'] = df['종가'].ewm(span=short_window, adjust=False).mean()
            df['EMA_long'] = df['종가'].ewm(span=long_window, adjust=False).mean()
            df['MACD'] = df['EMA_short'] - df['EMA_long']
            df['MACD_Signal'] = df['MACD'].ewm(span=signal_window, adjust=False).mean()
            return df

        def calculate_rsi(df, window=14, signal_window=9):
            """RSI 및 RSI 신호 계산"""
            delta = df['종가'].diff()
            gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()
            loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()

            rs = gain / loss
            df['RSI'] = 100 - (100 / (1 + rs))
            
            # RSI 신호선 추가
            df['RSI_Signal'] = df['RSI'].ewm(span=signal_window, adjust=False).mean()
            return df

        # MACD 및 MACD 신호 계산
        df = calculate_macd(df)

        df = calculate_rsi(df)

        # '종가' 컬럼이 포함된 DataFrame을 가정합니다. 예를 들어, df라고 합시다.
        # df = pd.read_csv('path_to_your_data.csv') # 데이터 파일을 로드하는 예시

        # 지정된 기간에 대한 SMA 계산
        periods = [5, 20, 60, 120, 240]
        for period in periods:
            df[f'SMA_{period}'] = df['종가'].rolling(window=period).mean()

        # 지정된 기간에 대한 EMA 계산
        for period in periods:
            df[f'EMA_{period}'] = df['종가'].ewm(span=period, adjust=False).mean()

        for i in range(len(periods)):
            for j in range(i + 1, len(periods)):
                smaller_period = periods[i]
                larger_period = periods[j]
                df[f'SMA_{smaller_period}_minus_SMA_{larger_period}'] = df[f'SMA_{smaller_period}'] - df[f'SMA_{larger_period}']

        # EMA 간의 차이 계산
        for i in range(len(periods)):
            for j in range(i + 1, len(periods)):
                smaller_period = periods[i]
                larger_period = periods[j]
                df[f'EMA_{smaller_period}_minus_EMA_{larger_period}'] = df[f'EMA_{smaller_period}'] - df[f'EMA_{larger_period}']

        for column in ['시가', '고가', '저가','거래량','MACD','MACD_Signal','RSI','RSI_Signal']:
            df[f'{column}_등락률'] = df[column].pct_change() * 100

        # SMA 및 EMA의 전 거래일 대비 등락률 계산 및 DataFrame에 추가
        periods = [5, 20, 60, 120, 240]

        # SMA 등락률 계산 및 추가
        for period in periods:
            df[f'SMA_{period}_등락률'] = df[f'SMA_{period}'].pct_change() * 100

        # EMA 등락률 계산 및 추가
        for period in periods:
            df[f'EMA_{period}_등락률'] = df[f'EMA_{period}'].pct_change() * 100

        # 'MACD'가 0 이상일 때 1, 아니면 0을 할당
        df['MACD_Binary'] = (df['MACD'] >= 0).astype(int)

        # 'MACD - MACD_Signal'이 0 이상일 때 1, 아니면 0을 할당
        df['MACD_minus_Signal_Binary'] = ((df['MACD'] - df['MACD_Signal']) >= 0).astype(int)

        # 'RSI - RSI_Signal'이 0 이상일 때 1, 아니면 0을 할당
        df['RSI_minus_Signal_Binary'] = ((df['RSI'] - df['RSI_Signal']) >= 0).astype(int)

        # 시가 대비 종가 등락률 컬럼 추가
        df['시가_대비_종가_등락률'] = ((df['종가'] - df['시가']) / df['시가']) * 100
        df['시가_대비_저가_등락률'] = ((df['저가'] - df['시가']) / df['시가']) * 100
        df['시가_대비_고가_등락률'] = ((df['고가'] - df['시가']) / df['시가']) * 100
        df['저가_대비_종가_등락률'] = ((df['종가'] - df['저가']) / df['저가']) * 100
        df['저가_대비_고가_등락률'] = ((df['고가'] - df['저가']) / df['저가']) * 100
        df['고가_대비_종가_등락률'] = ((df['종가'] - df['고가']) / df['고가']) * 100
        # 종가 - SMA [5, 20, 60, 120, 240] 값의 차이 컬럼 추가
        for period in [5, 20, 60, 120, 240]:
            df[f'종가_minus_SMA_{period}'] = df['종가'] - df[f'SMA_{period}']

        # 종가 - EMA [5, 20, 60, 120, 240] 값의 차이 컬럼 추가
        for period in [5, 20, 60, 120, 240]:
            df[f'종가_minus_EMA_{period}'] = df['종가'] - df[f'EMA_{period}']


        #csv_file_path = "C:/apps/h1/이스트소프트_data.csv"  # 저장할 파일 경로 및 이름 설정
        #df.to_csv(csv_file_path, encoding='utf-8-sig')

        #print(f"Data saved to {csv_file_path}")

        # 등락률을 기준으로 다음 날 등락률 계산 후 target 생성
        df['next_day_return'] = df['등락률'].shift(-1)
        df['target'] = df['next_day_return'].apply(lambda x: 1 if x > 0.25 else -1)
        df.dropna(inplace=True)  # 마지막 행 삭제

        first_column_name = df.columns[0]

        
        X = df.drop(['next_day_return','target'], axis=1)
        y = df['target']

        # Calculating correlations for Forward Selection
        correlations = X.corrwith(y).abs().sort_values(ascending=False).reset_index()
        correlations.columns = ['Feature', 'Correlation']

        top_8_features = correlations.sort_values(by='Correlation', ascending=False).head(8)['Feature']

        # X와 y 정의 (df2와 이전 코드에서의 처리 과정을 바탕으로)
        X_top8 = X[top_8_features]
        y = df['target'] # y 값을 0과 1로 조정

        # 데이터를 학습 세트와 테스트 세트로 분할
        X_train, X_test, y_train, y_test = train_test_split(X_top8, y, test_size=0.25, random_state=42)

        # 모델 초기화
        rf_model = RandomForestClassifier()
        dt_model = DecisionTreeClassifier()
        lr_model = LogisticRegression()
        svm_model = SVC()

        # 모델 학습
        rf_model.fit(X_train, y_train)
        dt_model.fit(X_train, y_train)
        lr_model.fit(X_train, y_train)
        svm_model.fit(X_train, y_train)

        # 예측
        y_pred_rf = rf_model.predict(X_test)
        y_pred_dt = dt_model.predict(X_test)
        y_pred_lr = lr_model.predict(X_test)
        y_pred_svm = svm_model.predict(X_test)

        # Precision 계산
        precision_rf = precision_score(y_test, y_pred_rf, average='macro')
        precision_dt = precision_score(y_test, y_pred_dt, average='macro')
        precision_lr = precision_score(y_test, y_pred_lr, average='macro')
        precision_svm = precision_score(y_test, y_pred_svm, average='macro')

        # 결과 저장
        results_list.append({
            'Ticker': ticker,
            'Name': ticker_name,
            'Precision_RF': precision_rf,
            'Precision_DT': precision_dt,
            'Precision_LR': precision_lr,
            'Precision_SVM': precision_svm
        })
        pass
    except Exception as e:
        print(f"Error processing {ticker}: {e}")
        continue

# 결과를 데이터프레임으로 변환하고 precision_svm 값에 따라 내림차순 정렬
results_df = pd.DataFrame(results_list).sort_values(by='Precision_SVM', ascending=False)



Processing:   1%|          | 17/1721 [00:20<18:27,  1.54it/s] 

Error processing 456440: With n_samples=0, test_size=0.25 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.


Processing:   2%|▏         | 32/1721 [00:32<16:21,  1.72it/s]

Error processing 440290: With n_samples=0, test_size=0.25 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.


Processing:   3%|▎         | 47/1721 [00:43<10:35,  2.63it/s]

...

Processing:  97%|█████████▋| 1674/1721 [21:47<00:31,  1.47it/s]

Error processing 455310: With n_samples=0, test_size=0.25 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.


Processing:  98%|█████████▊| 1691/1721 [22:00<00:18,  1.59it/s]

Error processing 460930: With n_samples=0, test_size=0.25 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.


Processing: 100%|██████████| 1721/1721 [22:25<00:00,  1.28it/s]


In [4]:
import pandas as pd

# 전체 행을 출력하도록 설정
pd.set_option('display.max_rows', None)

# 전체 열을 출력하도록 설정
pd.set_option('display.max_columns', None)

results_df

,Ticker,Name,Precision_RF,Precision_DT,Precision_LR,Precision_SVM
510,448740,삼성스팩8호,1.000000,1.000000,1.000000,1.000000
44,099520,ITX-AI,1.000000,0.500000,1.000000,1.000000
1474,446750,하나26호스팩,0.500000,0.500000,0.500000,1.000000
871,438580,엔에이치스팩25호,1.000000,1.000000,1.000000,1.000000
859,356680,엑스게이트,0.500000,0.500000,0.500000,1.000000
1136,178780,일월지엠엘,1.000000,1.000000,1.000000,1.000000
1229,362320,청담글로벌,0.647436,0.509046,0.578049,0.870000
1441,335810,프리시젼바이오,0.483600,0.525253,0.506361,0.846715
236,006580,대양제지,0.575000,0.623626,0.552941,0.846154
146,036190,금화피에스시,0.504505,0.489583,0.494204,0.838710
